#### 以train数据集作为全部数据集使用以下模型进行建模

#### 数据集预处理
    -读取，分词，去除停用词
    -切分数据集
    -向量化

In [1]:
import pandas as pd
import jieba
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('./cnews/cnews.train.txt', sep='\t', encoding='utf-8', header=None, names=['label','content'])
data.head(1)

,label,content
0,体育,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...


In [3]:
#停用词
stop_words = pd.read_csv('./cnews/stopwords.txt', index_col = False, quoting=3, sep='\t',
                        names = ['stopword'], encoding='utf-8')
stop_words.head(2)

,stopword
0,!
1,""""


In [4]:
content = data.content.values.tolist()
label = data.label.values.tolist()
stopword = stop_words.stopword.values.tolist()

In [5]:
#去除停用词
def preprocess_text(content,label,result):
    for i in range(len(content)):
        try:
            segs = jieba.lcut(content[i])
            segs = filter(lambda x: len(x)>1, segs)
            segs = filter(lambda x: x not in stopword, segs)
            result.append((" ".join(segs), label[i]))
        except:
            print(content[i])
            continue

In [6]:
result = []
preprocess_text(content,label,result)
result[:1]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sr/qrwj9nk92555240wtx2w98wh0000gn/T/jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built succesfully.


[('马晓旭 意外 受伤 国奥 警惕 无奈 大雨 青睐 殷家 傅亚雨 沈阳 报道 来到 沈阳 国奥队 依然 摆脱 雨水 困扰 国奥队 日常 训练 再度 大雨 干扰 无奈 之下 队员 慢跑 分钟 草草收场 国奥队 奥体中心 外场 训练 阴沉沉 气象预报 显示 当天 沈阳 大雨 幸好 队伍 训练 干扰 球队 抵达 训练场 大雨 几个 小时 丝毫 停下来 试一试 态度 球队 当天 例行 训练 分钟 天气 转好 迹象 保护 球员 国奥队 中止 当天 训练 全队 返回 酒店 训练 足球队 稀罕 奥运会 全队 变得 娇贵 沈阳 一周 训练 国奥队 保证 现有 球员 出现意外 伤病 情况 影响 正式 比赛 这一 阶段 控制 训练 受伤 控制 感冒 疾病 队伍 放在 位置 抵达 沈阳 后卫 冯萧霆 训练 冯萧霆 长春 患上 感冒 参加 塞尔维亚 热身赛 队伍 介绍 冯萧霆 发烧 症状 两天 静养 休息 感冒 恢复 训练 冯萧霆 例子 国奥队 对雨中 训练 显得 特别 谨慎 担心 球员 受凉 引发 感冒 非战斗 减员 女足 队员 马晓旭 热身赛 受伤 导致 无缘 奥运 前科 沈阳 国奥队 警惕 训练 嘱咐 队员 动作 再出 事情 一位 工作人员 长春 沈阳 雨水 一路 伴随 国奥队 长春 几次 训练 大雨 搅和 没想到 沈阳 碰到 事情 一位 国奥 球员 雨水 青睐 不解',
  '体育')]

In [7]:
# 打乱顺序，生成训练集
random.shuffle(result)

x,y = zip(*result)
#把原始数据集切分为训练集和测试集
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1)

print(np.shape(x_train),np.shape(x_test))

(37500,) (12500,)


In [14]:
vec = CountVectorizer(analyzer='word', 
                        ngram_range=(1,3), #ngram=1,2,3
                        max_features=2000)
vec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=2000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

#### 贝叶斯分类器

In [15]:
from sklearn.naive_bayes import MultinomialNB
classfier = MultinomialNB()
classfier.fit(vec.transform(x_train),y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
#准确率
classfier.score(vec.transform(x_test),y_test)

0.92312

#### SVM

[参数说明](https://m.imooc.com/article/details?article_id=40903)

https://blog.csdn.net/qq_37879432/article/details/78598995

In [17]:
from sklearn import svm
#svm（linear、rbf、sigmoid为核的SVM）
clf_linear = svm.SVC(kernel='linear')
# clf_rbf = svm.SVC(kernel='rbf')
# clf_sigmoid = svm.SVC(kernel='sigmoid')
clf_linear.fit(vec.transform(x_train),y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [18]:
clf_linear.score(vec.transform(x_test),y_test)

0.94096

#### 主题模型 LDA

In [10]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

In [24]:
texts = []

random.shuffle(content)
for i in content[:100]:
    seg = list(j for j in jieba.lcut(i) if str(j) not in stopword and len(j)>1)
    texts.append(seg)
    
dic = Dictionary(texts)
doc = [dic.doc2bow(i) for i in texts]

In [25]:
lda = LdaModel(doc,num_topics=5,id2word=dic,passes=1) 
#pass 遍历次数

In [26]:
lda.print_topics(num_topics=5,num_words=5)

[(0, '0.005*"中国" + 0.005*"电影" + 0.004*"项目" + 0.003*"经济" + 0.003*"汇率"'),
 (1, '0.008*"电影" + 0.005*"公司" + 0.004*"中国" + 0.004*"市场" + 0.003*"投资"'),
 (2, '0.004*"项目" + 0.003*"中国" + 0.003*"电影" + 0.003*"亿元" + 0.002*"球员"'),
 (3, '0.006*"中国" + 0.005*"项目" + 0.003*"基金" + 0.003*"投资" + 0.003*"市场"'),
 (4, '0.006*"基金" + 0.004*"投资" + 0.003*"债券" + 0.003*"亿元" + 0.003*"电影"')]